# **Evaluation**

A critical step in an ML workflow is evaluating your model. There are two important considerations when evaluating a classifier:

1. How to sample your data for testing and
2. What measurements to use.

We'll talk about both of these issues below, after a brief discussion about the different types of errors and limitations of _accuracy_ as a metric for evaluating ML models. 


### **Training Error vs. Generalization Error**

![](img/IST707-Week21.jpg)

In machine learning, evaluating a model's performance involves understanding two key concepts: training error and generalization error. **Training error** refers to the error that the model makes on the training data, the same data it learns from. It's a measure of how well the model fits the training data. However, fitting the training data too closely can lead to overfitting, where the model captures noise along with the underlying data pattern. This is where **generalization error** comes into play. Generalization error measures how well a model performs on unseen data, that is, data not used during the training process. It's an indicator of how well the model has learned to generalize from the training data to broader, unseen instances. A model that performs well on training data but poorly on unseen data (high generalization error) is overfitted, whereas a model that achieves a balance, performing well both on training and unseen data, is considered well-generalized.

#### Example
Consider a model trained to predict house prices. If this model is trained on a specific dataset of house prices in a city and achieves very low error rates on this training set, it has a low training error. However, if this model, when used to predict prices of houses in a different city (unseen data), performs poorly, it has a high generalization error. The goal in model development is to minimize both training error and generalization error to create a model that is accurate and robust against unseen data.



### **Accuracy is Not Always Enough**

**Accuracy** is a commonly used metric in classification problems which calculates the proportion of correct predictions out of all predictions made. It's defined as the number of correct predictions (both true positives and true negatives) divided by the total number of predictions.

However, accuracy can be misleading, especially in cases where the dataset is imbalanced, meaning there's a significant difference in the number of instances of the different classes.

#### Limitations of Accuracy
1. **Does Not Reflect Class Imbalance**: Accuracy does not take into account the imbalance in the distribution of the classes. In a highly imbalanced dataset, even a naive model predicting only the majority class can yield a high accuracy.
2. **No Insight into Type of Errors**: Accuracy does not distinguish between the types of errors (false positives vs. false negatives), which can be critical in certain domains like medical diagnosis or fraud detection.

#### Example: Breast Cancer Testing
Consider a breast cancer screening test applied to 1000 individuals with an accuracy of 94.5%, and the base rate of breast cancer in this population is 5%.

- Out of 1000 individuals, 50 (5%) have breast cancer, and 950 (95%) do not.
- With an accuracy of 94.5%, the test correctly identifies 945 individuals (both cancer and non-cancer cases).

Let's break down this accuracy:

1. **True Positives (TP)**: These are individuals who have breast cancer and are correctly identified by the test as having it.
2. **True Negatives (TN)**: These are individuals who do not have breast cancer and are correctly identified by the test as not having it.
3. **False Positives (FP)**: These are individuals who do not have breast cancer but are incorrectly identified by the test as having it.
4. **False Negatives (FN)**: These are individuals who have breast cancer but are incorrectly identified by the test as not having it.

Given the accuracy and base rate, we can't directly deduce the exact number of TP, FP, TN, and FN, but we can infer the following:

- If all 50 actual cancer cases are correctly identified, then we have 50 TPs. The remaining 895 correct predictions must be TNs (945 correct predictions minus 50 TPs). This would leave 55 incorrect predictions (1000 total minus 945 correct predictions), which would all have to be FPs since all the TPs are accounted for. This scenario implies a very high false positive rate.
- Conversely, if some of the cancer cases are missed (FNs), the number of FPs would decrease, but this would increase the FN rate, which is also problematic.

Thus, even with a high accuracy of 94.5%, the low base rate of cancer significantly affects the interpretation of the result. A high number of false positives or false negatives can still occur, which can be critical in a medical context. This example illustrates why accuracy alone, especially in the context of imbalanced datasets, might not be a sufficient metric for evaluating the performance of a diagnostic test. Other metrics like precision, recall, and the F1 score are crucial for a more complete evaluation.



## **Sampling Data**

With a basic understanding of some of the issues underlying evaluation, we can now turn to the challenge of how to sample data for purposes of evaluation.

## Train-Test Split

* __Basic Idea__ :  Separate your data into two sets – one for “training” the model\, and the other for “testing” the model
* __Important Considerations__
  * The training set must be  _completely independent_ and  _highly representative_ of the test set to get an unbiased estimate of performance
  * Performance can vary depending on the split\, and a bad\-split could result in poorer performance
* __Examples__
  * Binary classification with a rare positive class \(e\.g\.\, disease detection where only 5% samples are positive\)
  * Time\-series data where future data points are used in the training\, and past data points in the training
  * Using data from one geographic region in the training data and another region in the test set

### Example

The following shows how to use `train_test_split` in sklearn.

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Load a built-in dataset (Iris dataset)
iris = load_iris()
X, y = iris.data, iris.target

# Example without a fixed random seed
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Train a simple model
model = LogisticRegression(max_iter=200)
model.fit(X_train, y_train)

# Predict and calculate accuracy
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

Note that the train test split is different every time, and so you might get different results!

In [ ]:
results = []
for _ in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    results.append(accuracy_score(y_test, y_pred))

results

For reproducibility, you can set a random seed to make sure the `random` engine produces the same split every time.

In [ ]:
# Let's set a seed

seed = 200

results = []
for _ in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    results.append(accuracy_score(y_test, y_pred))

results




### Stratification

Stratification is a technique used in data splitting, especially important in classification problems, to ensure that each subset of the data (such as training and testing sets) has a similar distribution of class labels as the original dataset. This is particularly crucial when dealing with imbalanced datasets, where one or more classes are underrepresented compared to others.

When performing a train-test split without stratification in an imbalanced dataset, there's a risk that the distribution of classes in the training and testing sets will be significantly different from each other and from the original dataset. This can lead to biased or inaccurate models, as the model may not be trained or evaluated on a representative sample of data.

Stratification addresses this issue by dividing the data in a way that maintains the same proportion of each class in both the training and testing sets as found in the original dataset. In Scikit-Learn's `train_test_split` function, this is achieved by setting the `stratify` parameter to the class labels. As a result, stratification leads to more reliable model evaluation and performance metrics, particularly in scenarios of class imbalance.



### Example

First, we're going to artificially unbalance scikit learn's built in breast cancer dataset.

In [ ]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.utils import shuffle

# Load the Breast Cancer dataset
data = load_breast_cancer()
X, y = data.data, data.target

# Separate the majority and minority classes
majority_class_X = X[y == 0]
majority_class_y = y[y == 0]
minority_class_X = X[y == 1]
minority_class_y = y[y == 1]

# Downsample the minority class
minority_size = int(0.1 * len(majority_class_y))  # 10% of the majority class size
downsampled_minority_X = minority_class_X[:minority_size]
downsampled_minority_y = minority_class_y[:minority_size]

# Combine the downsampled minority class with the majority class
X_downsampled = np.concatenate([majority_class_X, downsampled_minority_X])
y_downsampled = np.concatenate([majority_class_y, downsampled_minority_y])

# Shuffle the combined dataset
X_downsampled, y_downsampled = shuffle(X_downsampled, y_downsampled, random_state=42)

# Now X_downsampled and y_downsampled have the downsampled dataset

Now analyze results with the imbalanced data.

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score


# Function to perform train_test_split and evaluate the model
def evaluate_model(X, y, stratify=None):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=stratify)
    model = SGDClassifier(random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return accuracy_score(y_test, y_pred)

# Evaluating the model without stratification
acc_without_stratification = [evaluate_model(X_downsampled, y_downsampled) for _ in range(200)]

# Evaluating the model with stratification
acc_with_stratification = [evaluate_model(X_downsampled, y_downsampled, stratify=y_downsampled) for _ in range(200)]

# Display results
import matplotlib.pyplot as plt

# Plotting the results
plt.figure(figsize=(12, 6))

# Histogram for accuracies without stratification
plt.subplot(1, 2, 1)  # 1 row, 2 columns, 1st subplot
plt.hist(acc_without_stratification, bins=10, alpha=0.7, color='blue')
plt.title('Accuracies without Stratification')
plt.xlabel('Accuracy')
plt.ylabel('Frequency')

# Histogram for accuracies with stratification
plt.subplot(1, 2, 2)  # 1 row, 2 columns, 2nd subplot
plt.hist(acc_with_stratification, bins=10, alpha=0.7, color='green')
plt.title('Accuracies with Stratification')
plt.xlabel('Accuracy')
plt.ylabel('Frequency')

plt.tight_layout()  # Adjusts the plots to fit into the figure cleanly
plt.show()



If you run the preceding example several times, you should find that the accuracies with stratification have lower overall variance than those models trained and tested without stratification.


### Cross-validation

After observing that a single train-test split can lead to varying results, cross-validation emerges as a solution to gain a more stable and accurate estimate of a model's performance. In cross-validation, the dataset is divided into multiple smaller sets or "folds". The model is then trained and evaluated multiple times, with each fold getting a chance to serve as the test set while the remaining parts are used for training. The most common form of this technique is *k-fold cross-validation*, where the data is split into 'k' number of folds. For each iteration, a different fold is used for testing, and the rest for training. The final performance metric is typically the average of the model's performance across all folds.

This process helps in mitigating the variability that comes with relying on a single split. It ensures that every data point is used for both training and testing, which makes the evaluation more reliable and less dependent on the particular way the data is split. By using cross-validation, you gain a better understanding of how the model is likely to perform on unseen data, making it a staple technique in the machine learning model evaluation process.

* __K\-Fold Cross validation:__
  * Partition data into  _k_  subsets or "folds\."
  * Train on  _k_ −1 of these folds and test on the remaining fold
  * Repeat this process  _k_  times\, average performance metrics
* __Leave\-one\-out:__
  * Extreme cross\-validation \- train on all available data\, holding back just one case for testing
  * Computationally very expensive
* __Stratified K\-Fold Cross\-Validation:__
  * Each fold is stratified
* __Considerations__ :
  * Not appropriate for time\-series data \(use time\-series specific cross\-validation\)
  * A greater number of folds increases available training data but
  * Increases processing time and performance variance
  * Reduces representativeness of test samples
  * Data leakage if any preprocessing / feature selection is done after splitting but before training\.  All such operations need to take place on the training set\.


### Cross-validation code

Scikit learn has a robust API for handling cross-validation, which offers a lot of flexibility.  The following offers a quick introduction.

#### **Interlude: Preparing the data**

For the following, we'll use a dataset of handwritten digits, commonly used to demonstrate ML algorithms.

In [ ]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', as_frame=False)

In [ ]:
import pprint
pp = pprint.PrettyPrinter(width=120)
pp.pprint(mnist.DESCR)

In [ ]:
mnist.keys()  # extra code – we only use data and target in this notebook

Note, data and target exist for most sklearn datasets.  Data is a matrix of features, and target is a vector of labels.

In [ ]:
X, y = mnist.data, mnist.target
X

If you wanted to see these as a DataFrame, you could do this:

In [ ]:
import pandas as pd
df = pd.DataFrame(mnist.data)
df["target"] = mnist.target
df

But that's not really necessary here.  We'll just use these things as matrices and arrays.  Let's look at the shape here.

In [ ]:
print(f"X shape = {X.shape}")
print(f"y shape = {y.shape}")
print(f"Is number of rows equal? {X.shape[0] == y.shape[0]}")

Where does the 784 come from?  Read the docs above!

In [ ]:
28 * 28

Let's take a look at one of these images

In [ ]:
import matplotlib.pyplot as plt

# Define function to plot a single digit
def plot_digit(image_data):
    image = image_data.reshape(28, 28)
    plt.imshow(image, cmap="binary")
    plt.axis("off")

some_digit = X[0]
plot_digit(some_digit)
plt.show()

What is it?

In [ ]:
y[0]

Here's another sample:

In [ ]:
plt.figure(figsize=(9, 9))
for idx, image_data in enumerate(X[:100]):
    plt.subplot(10, 10, idx + 1)
    plot_digit(image_data)
plt.subplots_adjust(wspace=0, hspace=0)
plt.show()

### Training the classfier

Let's just go with predicting '5s'.

In [ ]:
from sklearn.preprocessing import StandardScaler
# Setting up the data here
# Though it's not really necessary here (MINST variables are all roughly the same) it's often useful to z-score
# your data.  Especially important for logistic regression
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)

# Arbitratily taking the first 60000 rows here to reduce processing time
X, y = X[:60000], y[:60000]

y_5 = (y == '5')  # True for all 5s, False for all other digits

# Save for later
digit_data = X,y_5

In [ ]:
# I'm going to use the SGDClassifier here - it's a lot like a logistic regression with an "sag" solver,
# but is a lot faster with larger data sets

from sklearn.linear_model import SGDClassifier

digit_clf = SGDClassifier(random_state=42)
digit_clf.fit(X, y_5)

In [ ]:
# Recall we already checked that X[0] is a five
some_digit = X[0]

# Note that predict expect an array of values, so we tuck this into an array before testing.
digit_clf.predict([some_digit])

#### **Using cross_val_score**

`cross_val_score` is an easy one liner that handles most of the simple cases for cross validation.  In the following, we use three folds.

In [ ]:
from sklearn.model_selection import cross_val_score

sgd_clf = SGDClassifier()
# Scikit learn makes it easy to use cross validation with simple measures
# CV is the number of folds
cross_val_score(sgd_clf, X, y_5, cv=3, scoring="accuracy")


Note that `cross_val_score` only reports one measure (passed as the 'scoring' parameter).  If you want to pass more than one measure, you can use the `cross_validate` method.  Note, we'll talk about the different measures later.

In [ ]:
from sklearn.metrics import make_scorer
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import cross_validate

scoring = {
    'f1': 'f1_macro',
    'precision': make_scorer(precision_score, average='macro'),
    'recall': make_scorer(recall_score, average='macro')
}

# Using multiple metrics
scores = cross_validate(sgd_clf, X, y_5, cv=3, scoring=scoring,return_train_score=True)
for key, values in scores.items():
    print(f"{key}: {values.mean():.3f} (+/- {values.std() * 2:.3f})")


#### **KFold sampler**

If you want even more control over your training and testing, you can use the `KFold` class in sklearn.

In [ ]:
from sklearn.model_selection import KFold
from sklearn.base import clone
from sklearn.metrics import accuracy_score
kf = KFold(n_splits=3, random_state=42, shuffle=True)
# Initialize Logistic Regression and StandardScaler
clf = SGDClassifier()
scaler = StandardScaler()

# Initialize list to store accuracy for each fold
accuracy_list = []

# Loop through each fold
for train_index, test_index in kf.split(X_train):
    # Split the data into current train and test set
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y_5[train_index], y_5[test_index]

    # It's a good idea to use a fresh, untrained model each time you run on new data
    # The "clone" command does that, but simplifies things by copying other parameters

    clone_clf = clone(clf)

    # Fit the model
    clf.fit(X_train, y_train)

    # Make predictions
    y_pred = clf.predict(X_test)

    # Calculate accuracy
    acc = accuracy_score(y_test, y_pred)
    accuracy_list.append(acc)

# Calculate and print the average accuracy
average_accuracy = sum(accuracy_list) / len(accuracy_list)
print(f'Average Accuracy: {average_accuracy}')

#### **Stratified K-Fold Sampling**

If we want to run a stratified K-Fold sampler, we can use the `StratifiedKFold` class.  


In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone
clf = SGDClassifier()

skfolds = StratifiedKFold(n_splits=3)  # add shuffle=True if the dataset is not
                                       # already shuffled
for train_index, test_index in skfolds.split(X, y_5):
    clone_clf = clone(sgd_clf)
    X_train_folds = X[train_index]
    y_train_folds = y_5[train_index]
    X_test_fold = X[test_index]
    y_test_fold = y_5[test_index]

    clone_clf.fit(X_train_folds, y_train_folds)
    y_pred = clone_clf.predict(X_test_fold)
    n_correct = sum(y_pred == y_test_fold)
    print(n_correct / len(y_pred))



Note that if we don't need fine-grained control, we can simply pass the sampler into the `cross_val_score` method.


In [ ]:
from sklearn.model_selection import StratifiedKFold

# Create a stratified K-Fold object
stratified_kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Use cross_val_score with stratified K-Fold
clf = SGDClassifier()
scores = cross_val_score(clf, X, y_5, cv=stratified_kfold, scoring='accuracy')
scores

#### **Working with pipelines**

Note that combining scikit learn's cross validation routines with pipelines simplify the process of data preparation while avoiding data leakage.  

As a working example, we'll use the `wine` dataset, which is focused on predicting the quality of wine based on it's chemical makeup.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Load the dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
wine_data = pd.read_csv(url, delimiter=";")
wine_data

We'll divide wines into `good` quality and `bad` quality by selecting all wines with a quality score of 7 or more.

In [ ]:
# Create binary classification target variable
wine_data['quality_label'] = wine_data['quality'].apply(lambda x: 1 if x >= 7 else 0)

# Features and Target
X = wine_data.drop(['quality', 'quality_label'], axis=1)
y = wine_data['quality_label']

wine_training_data = X,y


Note that the wine dataset features are of different orders of magnitude, so we'll want to scale features before training.

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

# Create a pipeline with StandardScaler and LogisticRegression
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Step 1: Scale the data
    ('classifier', LogisticRegression())  # Step 2: Train a logistic regression model
])

Finally, the `Pipeline` class follows the estimator API, so all we have to do is pass it into `cross_val_score`.  This will take care of running scaling separately on each fold, hence avoiding any data leakage problems.

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

# Create a pipeline with StandardScaler and LogisticRegression
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Step 1: Scale the data
    ('classifier', LogisticRegression())  # Step 2: Train a logistic regression model
])
# Use cross_val_score to get the scores for each fold
scores = cross_val_score(pipeline, X, y, cv=5)

print("Cross-validation scores:", scores)
print("Mean cross-validation score:", scores.mean())

#### **Leave One Out evaluation**

Finally, sklearn provide a `LeaveOneOut` sampler. Keep in mind that LOO can be computationally expensive for large datasets because it will train a new model for each sample in the dataset. It's generally used for small datasets or for cases where a high-variance estimate is acceptable.  Note, we'll only do this with a sample dataset, because it is computationally expensive to run.  

In [ ]:
from sklearn.model_selection import LeaveOneOut, cross_val_score
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression

# Create a dataset
X, y = make_classification(n_classes=2, class_sep=2,
weights=[0.1, 0.9], n_informative=3, n_redundant=0,
flip_y=0, n_features=20, n_clusters_per_class=1,
n_samples=100, random_state=42)

# Create a Leave-One-Out object
loo = LeaveOneOut()

# Use cross_val_score with Leave-One-Out
clf = LogisticRegression()
scores = cross_val_score(clf, X, y, cv=loo, scoring='accuracy')

print(f"Mean Accuracy: {scores.mean():.2f}")

## **Metrics**

As we've discussed, accuracy is limited in several ways.  There are many other means for evaluating performance, and what is most meaningful depends on the domain problem.  Scikit learn has a robust set of metrics, and I encourage you to read through the documentation hosted at https://scikit-learn.org/stable/modules/model_evaluation.html.

In the following, we'll cover some of the more common approaches as well as there implementation in sklearn.

### **Confusion Matrices**

A  _confusion matrix_  is a table layout that allows visualization of the performance of an algorithm.  Though not as convenient as a single number, confusion matrices provides a great deal of information, especially in the case of binary classifiers.  The cells of a confusion matrix include the following values:

__True Positives \(TP\)__ : These are the correctly predicted positive observations\.

__True Negatives \(TN\)__ : These are the correctly predicted negative observations\.

__False Positives \(FP\)__ : Incorrectly predicted positive observations \(Type I error\)\.

__False Negatives \(FN\)__ : Incorrectly predicted negative observations \(Type II error\)\.

**Example**

Considers and online store that sells video games, and a model that predicts whether a visitor will buy a game or not.

|   | predictions |  |  |
| :-: | :-: | :-: | :-: |
| Ground Truth | buy_game = yes | buy_game = no | total |
| buy_game = yes | 6700 (TP) | 300 (FN) | 7000 |
| buy_game = no | 900 (FP) | 100 (TN) | 1000 |
| total | 7600 | 400 | 8000 |

In sklearn, we can easily run a confusion matrix as follows (using our digit data):

In [ ]:
# You've hopefully already found the sklearn metrics library
from sklearn.metrics import confusion_matrix

X = digit_data[0]
y = digit_data[1]


clf = SGDClassifier()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

clf.fit(X_train,y_train)
y_pred = clf.predict(X_test) 

cm = confusion_matrix(y_test, y_pred)
cm

### **Precision, Recall, and F1 Score**

Precision, Recall, and the F1-Score are alternative metrics used to evaluate the performance of classification models, especially in scenarios where classes are imbalanced. Understanding these metrics is key to interpreting the effectiveness of a model beyond just accuracy.

### Precision
- **What It Measures**: Precision quantifies the accuracy of the model in predicting positive instances. It's the ratio of true positives (correctly predicted positives) to the total number of predicted positives (both true positives and false positives).
- **Formula**: Precision = True Positives / (True Positives + False Positives)
- **Interpretation**: A high precision indicates that the model is reliable in its positive predictions, but it doesn’t tell us how many actual positives were missed.

### Recall (Sensitivity)
- **What It Measures**: Recall measures the model's ability to detect positive instances among all actual positives. It's the ratio of true positives to the actual positives in the data (both true positives and false negatives).
- **Formula**: Recall = True Positives / (True Positives + False Negatives)
- **Interpretation**: High recall means that the model is good at detecting positive instances, but it may also be including some false positives.

### F1-Score
- **What It Measures**: The F1-Score is the harmonic mean of precision and recall. It provides a single score that balances both the concerns of precision and recall in one number.
- **Formula**: F1 = 2 * (Precision * Recall) / (Precision + Recall)
- **Interpretation**: A high F1-Score suggests a balanced trade-off between precision and recall. It is particularly useful when you seek a balance between these two metrics and when the class distribution is imbalanced.

![](img/IST707-Week223.png)

\* By Walber \- Own work\, CC BY\-SA 4\.0\, https://commons\.wikimedia\.org/w/index\.php?curid=36926283

#### Example

|   | predictions |  |  |
| :-: | :-: | :-: | :-: |
| Ground Truth | buy_game = yes | buy_game = no | total |
| buy_game = yes | 6700 (TP) | 300 (FN) | 7000 |
| buy_game = no | 900 (FP) | 100 (TN) | 1000 |
| total | 7600 | 400 | 8000 |

Precision: 6700/7600 = \.88

Recall: 6700 / 7000 = \.96

F1\-Score:  2\*\(\.96\+\.88\)/\(\.96\*\.88\)=\.92

Precision: 100/400 = \.25

Recall: 100 / 1000 = \.1

F1\-Score:  2\*\(\.25\+\.1\)/\(\.25\*\.1\)=\.14

__Average F1 across classes = \.53__

#### **Calculating Precision, Recall and F1**

Continuing with our digit dataset, sklearn provides metrics that make it easy to calculate precision and recall, as follows.

In [ ]:
from sklearn.metrics import precision_score, recall_score

precision_score(y_test, y_pred) 

In [ ]:
# Same computation, using the confusion matrix above
# TP / (FP + TP)
cm[1, 1] / (cm[0, 1] + cm[1, 1])

In [ ]:
recall_score(y_test, y_pred)

In [ ]:
# TP / (FN + TP)
cm[1, 1] / (cm[1, 0] + cm[1, 1])

In [ ]:
from sklearn.metrics import f1_score

f1_score(y_test, y_pred)

In [ ]:
# Calculating by hand
cm[1, 1] / (cm[1, 1] + (cm[1, 0] + cm[0, 1]) / 2)

### **ROC Curves**


Many machine learning classifiers, especially binary classifiers, work by computing a decision score or probability for each instance. This score indicates the likelihood of an instance belonging to a particular class. To make a final classification decision (e.g., class 0 or class 1), a threshold is set. Instances with scores above this threshold are assigned to one class, while those below are assigned to the other.

For example, in a logistic regression classifier, a decision score is calculated for each instance, often in the form of a probability. By default, a threshold of 0.5 is typically used: instances with probabilities above 0.5 are classified as the positive class, and those below as the negative class.

### The Precision-Recall Tradeoff
The choice of threshold has a direct impact on precision and recall, leading to their tradeoff:

- **Lowering the Threshold**: This increases recall but can decrease precision. By lowering the threshold, more instances are classified as positive, which means more actual positives are correctly identified (higher recall). However, this also leads to more false positives (lower precision).
  
- **Raising the Threshold**: Conversely, increasing the threshold boosts precision but can lower recall. A higher threshold means that only instances with a high likelihood are classified as positive, leading to fewer false positives (higher precision). However, this might result in missing out on actual positives (lower recall).



### Visualizing the tradeoff

In [ ]:
# The "decision_function" method returns the raw value, of the predictor, which is then 
# thresholded to achieve an outcome

y_scores = digit_clf.decision_function([some_digit])
y_scores

In [ ]:
from sklearn.model_selection import cross_val_predict
# We can plug this directly into cross_val_predict to get the scores across all of our data
y_scores = cross_val_predict(digit_clf, X, y, cv=3,
                             method="decision_function")

In [ ]:
# Scikit learn gives us a really nice way to look at this!
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
threshold = 3000
precisions, recalls, thresholds = precision_recall_curve(y_train_5, y_scores)
plt.figure(figsize=(8, 4))  # extra code – it's not needed, just formatting
plt.plot(thresholds, precisions[:-1], "b--", label="Precision", linewidth=2)
plt.plot(thresholds, recalls[:-1], "g-", label="Recall", linewidth=2)
plt.vlines(threshold, 0, 1.0, "k", "dotted", label="threshold")

idx = (thresholds >= threshold).argmax()  # first index ≥ threshold
plt.plot(thresholds[idx], precisions[idx], "bo")
plt.plot(thresholds[idx], recalls[idx], "go")
plt.axis([-50000, 50000, 0, 1])
plt.grid()
plt.xlabel("Threshold")
plt.legend(loc="center right")


plt.show()

In [ ]:
# We can graph these two together:

import matplotlib.patches as patches  # extra code – for the curved arrow

plt.figure(figsize=(6, 5))  # extra code – not needed, just formatting

plt.plot(recalls, precisions, linewidth=2, label="Precision/Recall curve")

plt.plot([recalls[idx], recalls[idx]], [0., precisions[idx]], "k:")
plt.plot([0.0, recalls[idx]], [precisions[idx], precisions[idx]], "k:")
plt.plot([recalls[idx]], [precisions[idx]], "ko",
         label="Point at threshold 3,000")
plt.gca().add_patch(patches.FancyArrowPatch(
    (0.79, 0.60), (0.61, 0.78),
    connectionstyle="arc3,rad=.2",
    arrowstyle="Simple, tail_width=1.5, head_width=8, head_length=10",
    color="#444444"))
plt.text(0.56, 0.62, "Higher\nthreshold", color="#333333")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.axis([0, 1, 0, 1])
plt.grid()
plt.legend(loc="lower left")

plt.show()

In [ ]:
# With this analysis, we can arbitrarily obtain a threshold to achieve a given level of precision or recall:

idx_for_90_precision = (precisions >= 0.90).argmax()
threshold_for_90_precision = thresholds[idx_for_90_precision]
threshold_for_90_precision



### Balancing the Tradeoff
- **Context-Dependent**: The optimal balance between precision and recall is highly dependent on the specific context or application. For instance, in spam email detection (where false positives are more tolerable than false negatives), a lower threshold might be preferable. In contrast, for medical diagnostics (where missing a positive diagnosis can be critical), a higher threshold might be chosen to ensure high recall.
  
- **Adjusting the Threshold**: Some classifiers allow manual adjustment of the decision threshold. Experimenting with different thresholds can help in finding the balance that best suits the specific needs and priorities of the task at hand.

### ROC and AUC

Receiver Operating Characteristic (ROC) curves and the Area Under the Curve (AUC), are tools that examine the tradeoff between precision and recall. These are particularly useful for evaluating classification models in the presence of class imbalance or when dealing with probabilistic predictions.

#### ROC Curves
- **What It Is**: Like the precision-recall curve above, the ROC curve is a graphical representation of a classifier's performance across all possible thresholds. It plots two parameters: 
   - **True Positive Rate (TPR)**, also known as Recall, on the y-axis.
   - **False Positive Rate (FPR)** on the x-axis.
- **TPR vs. FPR**: TPR (Recall) is calculated as True Positives / (True Positives + False Negatives), and FPR is calculated as False Positives / (False Positives + True Negatives).
- **Interpreting the Curve**: The ROC curve shows the tradeoff between sensitivity (or TPR) and specificity (1 - FPR). A higher curve indicates a better performance. An ideal classifier would have a curve that goes straight up the y-axis and then along the x-axis.

Sklearn provides tools to simplify visualization of ROC Curves.

In [ ]:
# SciKit learn also gives us ROC curves for free

from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(y_train_5, y_scores)

idx_for_threshold_at_90 = (thresholds <= threshold_for_90_precision).argmax()
tpr_90, fpr_90 = tpr[idx_for_threshold_at_90], fpr[idx_for_threshold_at_90]


plt.plot(fpr, tpr, linewidth=2, label="ROC curve")
plt.plot([0, 1], [0, 1], 'k:', label="Random classifier's ROC curve")
plt.plot([fpr_90], [tpr_90], "ko", label="Threshold for 90% precision")

# just beautifies the figure
plt.gca().add_patch(patches.FancyArrowPatch(
    (0.20, 0.89), (0.07, 0.70),
    connectionstyle="arc3,rad=.4",
    arrowstyle="Simple, tail_width=1.5, head_width=8, head_length=10",
    color="#444444"))
plt.text(0.12, 0.71, "Higher\nthreshold", color="#333333")
plt.xlabel('False Positive Rate (Fall-Out)')
plt.ylabel('True Positive Rate (Recall)')
plt.grid()
plt.axis([0, 1, 0, 1])
plt.legend(loc="lower right", fontsize=13)

plt.show()

Note that in the above example, we have enough data that the ROC curve appears to be smooth, but this is not always the case.  For example:

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt
import numpy as np

# Generate synthetic data
np.random.seed(0)
n_samples = 100

# Generate true labels (40% of them are positive)
y_true = np.random.choice([0, 1], size=n_samples, p=[0.8, 0.2])

# Generate predicted probabilities
y_score = np.linspace(0, 1, n_samples)
np.random.shuffle(y_score)

# Introduce some noise to make it more realistic
y_score = y_score + np.random.normal(0, 0.1, n_samples)
y_score = np.clip(y_score, 0, 1)

# Compute ROC curve and AUC score
fpr, tpr, _ = roc_curve(y_true, y_score)
roc_auc = roc_auc_score(y_true, y_score)

# Plot the ROC curve
plt.figure(figsize=(12,10))
plt.plot(fpr, tpr, color='darkorange', lw=1, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=1, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc='lower right')
plt.show()



### AUC (Area Under the ROC Curve)
AUC provides a single numeric metric to summarize the ROC curve. It represents the probability that the classifier will rank a randomly chosen positive instance higher than a randomly chosen negative one.
- **Interpretation**: 
   - An AUC of 1 indicates a perfect classifier.
   - An AUC of 0.5 suggests a no-skill classifier, equivalent to random guessing.
   - AUC values between 0.5 and 1 indicate different levels of classifier performance, with higher values signifying better classification.

#### Calculating the AUC

For probabilistic classifiers (not all classifiers are probabilistic!), you can use sklearn to calculate the AUC as follows:

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler

X = digit_data[0]
y = digit_data[1]


clf = LogisticRegression()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Step 1: Scale the data; important for Logistic regression
    ('classifier', LogisticRegression(max_iter=5000))  # Step 2: Train a logistic regression model
])

pipeline.fit(X_train,y_train)

# Get the fitted classifier
fitted_clf = pipeline.named_steps['classifier']
y_pred_prob = fitted_clf.predict_proba(X_test)[:, 1]

# Calculating AUC
auc = roc_auc_score(y_test, y_pred_prob)
print("AUC:", auc)

#### Using the Trapezoidal Rule

The AUC can also be computed manually using the trapezoidal rule, which is a method for estimating the area under a curve. This is particularly useful when you have the ROC curve points.

For example, if `(x[i], y[i])` and `(x[i+1], y[i+1])` are two consecutive points on the ROC curve, the area of the trapezoid between these points is given by:

$$\text{Area} = \frac{1}{2} \times (x[i+1] - x[i]) \times (y[i] + y[i+1])$$

To calculate the total AUC, sum up the areas of all such trapezoids formed by the consecutive points.

**Code Example**:
First, you need to obtain the TPR (True Positive Rate) and FPR (False Positive Rate) values at various thresholds, which can be done using `roc_curve` from Scikit-Learn. Then, apply the trapezoidal rule to these points.


In [ ]:
from sklearn.metrics import roc_curve

# Calculate FPR, TPR, and thresholds
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)

# Apply the trapezoidal rule to estimate AUC
auc_manual = np.trapz(tpr, fpr)
print("Manually Calculated AUC:", auc_manual)


**Explanation**:
- In the Scikit-Learn example, `predict_proba` is used to get the probability scores required for AUC calculation.
- In the manual calculation, `roc_curve` provides the FPR and TPR at various threshold levels, and `np.trapz` from NumPy applies the trapezoidal rule to these points to estimate the AUC.

Both methods give you the same (or very close) AUC value, illustrating the model's ability to distinguish between the classes.

#### Summary

ROC curves and AUC are powerful tools for evaluating and comparing classifiers, especially in scenarios where choosing an optimal threshold is challenging or when dealing with probabilistic predictions. They provide a comprehensive view of model performance across various thresholds and are robust against imbalances in the class distribution.

- **Threshold Independence**: Unlike precision and recall which depend on a specific threshold, ROC and AUC provide a threshold-independent measure of model performance. This is particularly useful when comparing different models.
- **Class Imbalance**: ROC curves are less sensitive to class imbalance compared to precision-recall curves. Even with an imbalanced dataset, the ROC curve can provide a meaningful representation of a model's ability to distinguish between classes.
